In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset


In [3]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
from skimage.color import rgba2rgb
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

#**Helper Functions**

In [4]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image = sample['image']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        return {'image': img }


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image  = sample['image']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image }


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image = sample['image']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image)}

#**Warping**

In [5]:
%cd /content/drive/MyDrive/FALdetector-master
from utils.tools import *
from utils.visualize import *
from PIL import Image
import sys
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset'

/content/drive/MyDrive/FALdetector-master
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset


In [6]:
class WarpDataset(Dataset):
    """Photoshop Detection dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.warp_dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.warp_dataset)

    def __getitem__(self, idx, find_face=False):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = os.path.join(self.root_dir,
                                self.warp_dataset.iloc[idx, 0])
        image = io.imread(img_path)
        if image.shape[2] == 4:
          image = rgba2rgb(image)
        im_w, im_h = Image.open(img_path).size
        if find_face:
          faces = face_detection(img_path, verbose=False, model_file='/content/drive/MyDrive/FALdetector-master/utils/dlib_face_detector/mmod_human_face_detector.dat')
          if len(faces) == 0:
              print("no face detected by dlib, exiting")
              sys.exit()
          face, box = faces[0]
          image = resize_shorter_side(face, 400)[0]

        labels = self.warp_dataset.iloc[idx, 2]
        labels = np.array([labels])
        labels = labels.astype('float')
        sample = {'image': image, 'labels': labels}

        if self.transform:
          sample = self.transform(sample)

        return sample['image'], labels

In [7]:
warp_dataset = WarpDataset(csv_file='PhotoshopDetectionDataset.csv', 
                           root_dir='./Collective',
                           transform=transforms.Compose([
                                               Rescale(512),
                                               RandomCrop(224),
                                               ToTensor()
                                        ]))

In [8]:
%cd ..

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project
